In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
import json

import sys
sys.path.append('..')
from input_pipeline import Pipeline

# Get images and boxes

In [ ]:
tf.reset_default_graph()

pipeline = Pipeline(
    ['/mnt/datasets/dan/CelebA/train_shards/shard-0000.tfrecords'],
    batch_size=100, image_size=(64, 64),
    num_landmarks=5, repeat=True, 
    shuffle=False, augmentation=True
)

dataset = pipeline.dataset
iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
init = iterator.make_initializer(dataset)
features, labels = iterator.get_next()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    I, L = sess.run([features, labels])

# Show an augmented image

In [ ]:
def draw_on_image(image, keypoints):

    image_copy = image.copy()
    draw = ImageDraw.Draw(image_copy, 'RGBA')
    
    for i, (y, x) in enumerate(keypoints):
        draw.ellipse([
            (x - 1.0, y - 1.0),
            (x + 1.0, y + 1.0)
        ], outline='red')
        draw.text((x, y), text=str(i), fill='blue')

    return image_copy

In [ ]:
# choose an image
i = 0
image = Image.fromarray((I[i]*255.0).astype('uint8'))
w, h = image.size
keypoints = L[i]*np.array([h, w])
draw_on_image(image, keypoints)